[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/observation_prediction_bodies.ipynb)

# Transduction from observation to prediction for bodies

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [1]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction_bodies.sh http://josd.github.io/eye/transduction/observation_prediction_bodies.sh
! curl -o observation_prediction_bodies.py http://josd.github.io/eye/transduction/observation_prediction_bodies.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample_bodies.observation http://josd.github.io/eye/transduction/sample_bodies.observation
! chmod +x observation_prediction_bodies.sh
#% rm -fr ~/t2t_data/observation_prediction_bodies
#% rm -fr ~/t2t_train/observation_prediction_bodies/transformer-transformer_small/

Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.5/dist-packages (1.6.1)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (0.98)
Requirement not upgraded as not directly required: flask in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (0.12.2)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.14.2)
Requirement not upgraded as not directly required: gunicorn in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (19.7.1)
Requirement not upgraded as not directly required: gym in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (0.9.5)
Requirement not upgraded as not directly required: sympy in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.1.1)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.5/dist-packages (from tensor2tensor) (1.11.0)
Requ

In [2]:
# See the observation_prediction_bodies problem

! pygmentize -g observation_prediction_bodies.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionBodies(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for bodies."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 50% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 6,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 4,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      # wind turbine size fa

In [3]:
# See the observation_prediction_bodies script

! pygmentize -g observation_prediction_bodies.sh

#!/bin/bash
PROBLEM=observation_prediction_bodies
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=3 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample_bodies.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample_bodies.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLE

In [4]:
# Run the observation_prediction_bodies script

! ./observation_prediction_bodies.sh

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module transduction from path /home/jdroo/github.com/josd/eye
[2018-05-03 17:34:06,859] Importing user module transduction from path /home/jdroo/github.com/josd/eye
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction_bodies
[2018-05-03 17:34:06,861] Generating problems:
    observation:
      * observation_prediction_bodies
INFO:tensorflow:Generating data for observation_prediction_bodies.
[2018-05-03 17:34:06,863] Generating data for observation_prediction_bodies.
INFO:tensorflow:Generating vocab file: /home/jdroo/t2t_data/observation_prediction_bodies/vocab.observation_prediction_bodies.16384.subwords
[2018-05-03 17:34:06,

Instructions for updating:
Please switch to tf.estimator.train_and_evaluate. You will also have to convert to a tf.estimator.Estimator.
[2018-05-03 17:34:17,732] From /usr/local/lib/python3.5/dist-packages/tensor2tensor/utils/trainer_lib.py:337: Experiment.__init__ (from tensorflow.contrib.learn.python.learn.experiment) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.estimator.train_and_evaluate. You will also have to convert to a tf.estimator.Estimator.
[2018-05-03 17:34:17,733] Experiment.continuous_train_and_eval (from tensorflow.contrib.learn.python.learn.experiment) is experimental and may change or be removed at any time, and without warning.
INFO:tensorflow:Training model for 100 steps
[2018-05-03 17:34:17,733] Training model for 100 steps
INFO:tensorflow:Reading data files from /home/jdroo/t2t_data/observation_prediction_bodies/observation_prediction_bodies-train*
[2018-05-03 17:34:17,754] Reading data files from /home/jdroo

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
[2018-05-03 17:35:38,228] Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-05-03 17:35:38,360] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
[2018-05-03 17:35:38,370] Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
[2018-05-03 17:35:40,981] Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Done calling model_fn.
[2018-05-03 17:35:41,757] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-03-15:35:41
[2018-05-03 17:35:41,777] Starting evaluation at 2018-05-03-15:35:41
INFO:tensorflow:Graph was finalized.
[2018-05-03 17:35:42,001] Graph was finalized.
2018-05-03 17:35:42.001965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu device

INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-05-03 17:37:00,997] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
[2018-05-03 17:37:01,007] Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
[2018-05-03 17:37:03,648] Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Done calling model_fn.
[2018-05-03 17:37:04,384] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-03-15:37:04
[2018-05-03 17:37:04,403] Starting evaluation at 2018-05-03-15:37:04
INFO:tensorflow:Graph was finalized.
[2018-05-03 17:37:04,780] Graph was finalized.
2018-05-03 17:37:04.780990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-05-03 17:37:04.781074: I tensorflow/core/common_runtime/gpu/gpu_device.cc:923] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-05-03 17:37:04.7

INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_754_256.bottom
[2018-05-03 17:38:23,615] Transforming feature 'inputs' with symbol_modality_754_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_754_256.targets_bottom
[2018-05-03 17:38:23,741] Transforming 'targets' with symbol_modality_754_256.targets_bottom
INFO:tensorflow:Building model body
[2018-05-03 17:38:23,753] Building model body
INFO:tensorflow:Transforming body output with symbol_modality_754_256.top
[2018-05-03 17:38:26,377] Transforming body output with symbol_modality_754_256.top
INFO:tensorflow:Done calling model_fn.
[2018-05-03 17:38:27,152] Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-03-15:38:27
[2018-05-03 17:38:27,171] Starting evaluation at 2018-05-03-15:38:27
INFO:tensorflow:Graph was finalized.
[2018-05-03 17:38:27,397] Graph was finalized.
2018-05-03 17:38:27.398097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu device

...

INFO:tensorflow:Running local_init_op.
[2018-05-03 17:56:47,947] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-03 17:56:48,001] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1601 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 17:56:56,209] Saving checkpoints for 1601 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.049881417, step = 1600
[2018-05-03 17:56:56,983] loss = 0.049881417, step = 1600
INFO:tensorflow:Saving checkpoints for 1700 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 17:57:48,213] Saving checkpoints for 1700 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.04732582.
[2018-05-03 17:57:48,968] Loss for final step: 0.04732582.
INFO:ten

INFO:tensorflow:Running local_init_op.
[2018-05-03 17:58:08,839] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-03 17:58:08,892] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1701 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 17:58:17,079] Saving checkpoints for 1701 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.03535888, step = 1700
[2018-05-03 17:58:17,816] loss = 0.03535888, step = 1700
INFO:tensorflow:Saving checkpoints for 1800 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 17:59:08,866] Saving checkpoints for 1800 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.032604966.
[2018-05-03 17:59:09,620] Loss for final step: 0.032604966.
INFO:ten

INFO:tensorflow:Running local_init_op.
[2018-05-03 17:59:29,687] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-03 17:59:29,740] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1801 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 17:59:37,812] Saving checkpoints for 1801 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.03710829, step = 1800
[2018-05-03 17:59:38,546] loss = 0.03710829, step = 1800
INFO:tensorflow:Saving checkpoints for 1900 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 18:00:29,647] Saving checkpoints for 1900 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.030719303.
[2018-05-03 18:00:30,408] Loss for final step: 0.030719303.
INFO:ten

INFO:tensorflow:Running local_init_op.
[2018-05-03 18:00:50,282] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[2018-05-03 18:00:50,335] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1901 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 18:00:58,405] Saving checkpoints for 1901 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:loss = 0.031002091, step = 1900
[2018-05-03 18:00:59,199] loss = 0.031002091, step = 1900
INFO:tensorflow:Saving checkpoints for 2000 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
[2018-05-03 18:01:52,288] Saving checkpoints for 2000 into /home/jdroo/t2t_train/observation_prediction_bodies/transformer-transformer_small/model.ckpt.
INFO:tensorflow:Loss for final step: 0.027268313.
[2018-05-03 18:01:53,143] Loss for final step: 0.027268313.
INFO:t

[2018-05-03 18:02:10,633] Input graph does not use tf.data.Dataset or contain a QueueRunner. That means predict yields forever. This is probably a mistake.
INFO:tensorflow:Calling model_fn.
[2018-05-03 18:02:10,633] Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'infer'
[2018-05-03 18:02:13,614] Setting T2TModel mode to 'infer'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
[2018-05-03 18:02:13,614] Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
[2018-05-03 18:02:13,614] Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
[2018-05-03 18:02:13,614] Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
[2018-05-03 18:02:13,614] Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
[2018-05-03 18:02:13,614] Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Beam Decoding with beam size 3
[2018

In [5]:
# See the transductions
# For each observation the top 3 predictions are shown with their respective log probability

! pygmentize -g sample_bodies.observation
print("->-")
! pygmentize -g sample_bodies.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.35	A_PERSON has BMI class O	-4.70	A_PERSON has BMI class N class N	-5.14
A_TURBINE producing 9533 kW	-0.43	A_TURBINE producing 9079 kW	-3.16	A_TURBINE producing 11916 kW	-4.33
